In [2]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 34.8 MB/s eta 0:00:00


In [3]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error


In [4]:
# The API I used did not require the use of an API Key in order to use the service


# Artist name for query
artist_name = "roy lichtenstein"

# API endpoint for searching artworks by Roy Lichtenstein
url = f"https://collectionapi.metmuseum.org/public/collection/v1/search?q={artist_name}"
response = requests.get(url)

# Creating an empty list to store DataFrames - reposnds to a 200 api response code
dfs = []

if response.status_code == 200:
    data = response.json()
    artwork_ids = data.get('objectIDs', [])

    # This is a loop that works through each artwork ID and extracts necessary fields from the artwork information
    for artwork_id in artwork_ids:
        artwork_url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{artwork_id}"
        artwork_response = requests.get(artwork_url)
        if artwork_response.status_code == 200:
            info = artwork_response.json()


            title = info.get('title', 'N/A')
            artist = artist_name
            medium = info.get('medium', 'N/A')
            dimensions = info.get('dimensions', 'N/A')
            classification = info.get('classification', 'N/A')
            year = info.get('objectDate', 'N/A')
            accessionYear = info.get('accessionYear', 'N/A')

            # Here I am creating a dataframe in pandas and appending to it by specifying the column names
            df = pd.DataFrame([[title, artist, medium, dimensions, classification, year, accessionYear]],
                              columns=['Title', 'Artist', 'Medium', 'Dimensions', 'Classification', 'Year', 'accessionYear'])
            dfs.append(df)
# Failure clause for testing
else:
    print("Failed to fetch artwork information from the Metropolitan Museum of Art API.")

# Concatenating all DataFrames in the list to a final DataFrame
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    print(final_df)
else:
    print("No artworks found.")


                           Title            Artist  \
0                        Galatea  roy lichtenstein   
1     Landscape with Philosopher  roy lichtenstein   
2            Untitled Still Life  roy lichtenstein   
3    Figures with Sunset (Study)  roy lichtenstein   
4    Reflections on Brushstrokes  roy lichtenstein   
..                           ...               ...   
101              The Love Letter  roy lichtenstein   
102    Autumn Rhythm (Number 30)  roy lichtenstein   
103        The Death of Socrates  roy lichtenstein   
104             A City on a Rock  roy lichtenstein   
105                        Woman  roy lichtenstein   

                                                Medium  \
0                         Painted and patinated bronze   
1                              Oil and magna on canvas   
2                                           Silkscreen   
3                Colored pencil with graphite on paper   
4    Lithograph, screenprint, woodcut, metalized pl...   
.. 

In [ ]:
# Checking final_df before committing to ETL pipeline
final_df.head()

,Title,Artist,Medium,Dimensions,Classification,Year,accessionYear
0,Galatea,roy lichtenstein,Painted and patinated bronze,89 3/4 x 29 x 19 in. (229.9 x 73.7 x 48.3 cm),Sculpture,1990,2003
1,Landscape with Philosopher,roy lichtenstein,Oil and magna on canvas,104 1/4 × 48 in. (264.8 × 121.9 cm),Paintings,1996,2015
2,Untitled Still Life,roy lichtenstein,Silkscreen,29 7/8 × 22 1/8 in. (75.9 × 56.2 cm),Prints,1997,1999
3,Figures with Sunset (Study),roy lichtenstein,Colored pencil with graphite on paper,12 1/16 × 8 5/8 in. (30.7 × 21.9 cm),Drawings,1978,1978
4,Reflections on Brushstrokes,roy lichtenstein,"Lithograph, screenprint, woodcut, metalized pl...",57 1/8 × 71 in. (145.1 × 180.3 cm),Prints,1990,2003


In [ ]:

# Creating a connection to DBeaver
connection = mysql.connector.connect(
    host= 'isba-dev-01.crw88eg88uar.us-east-1.rds.amazonaws.com',
    user= 'admin',
    password= 'isba_4715'
)

# Creating a cursor to execute queries
cursor = connection.cursor()

# Created a database named 'artworks' (should be sql_project in retrospect)

cursor.execute("CREATE DATABASE IF NOT EXISTS artworks")
cursor.execute("USE artworks")

# Create a table named 'roy_lichtenstein' to store the artwork information
cursor.execute("""
    CREATE TABLE IF NOT EXISTS roy_lichtenstein (
        Title VARCHAR(255),
        Artist VARCHAR(255),
        Medium VARCHAR(255),
        Dimensions VARCHAR(255),
        Classification VARCHAR(255),
        Year VARCHAR(255),
        accessionYear VARCHAR(255)
    )
""")

# Inserting the data from the final_df DataFrame into the table
for index, row in final_df.iterrows():
    cursor.execute("""
        INSERT INTO roy_lichtenstein (Title, Artist, Medium, Dimensions, Classification, Year, accessionYear)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (row['Title'], row['Artist'], row['Medium'], row['Dimensions'], row['Classification'], row['Year'], row['accessionYear']))

# Committing the changes to the database
connection.commit()

# Closing the cursor and connection
cursor.close()
connection.close()

print("Artwork information successfully inserted into the MySQL database.")


Artwork information successfully inserted into the MySQL database.
